In [20]:
from collections import defaultdict
from pathlib import Path

import gurobipy as gp
import polars as pl
from gurobipy import GRB


In [ ]:
def format_data(data_dir, discount_factor=5) -> pl.DataFrame:
    df = pl.read_csv(
        Path(f"{data_dir}/alternatives.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000
    )
    df = df.select(
        [
            "unit",
            "schedule",
            f"npv_{discount_factor}_percent",
            "stock_0",
            "stock_5",
            "stock_25",
            "harvest_value_5",
            "harvest_value_25",
            "stock_1_0",
            "stock_1_5",
            "stock_1_25",
            "stock_2_0",
            "stock_2_5",
            "stock_2_25",
            "stock_30_0",
            "stock_30_5",
            "stock_30_25",
        ]
    )
    df_keys = pl.read_csv(
        Path(f"{data_dir}/alternatives_key.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000
    )
    df_keys = df_keys.drop("holding")
    # Split the treatments into multiple columns, to make them easier to read or whatever
    df_keys = df_keys.with_columns(
        [
            pl.when(pl.col("treatment").str.contains("_5"))
            .then(pl.col("treatment").str.extract(r"(\w+)_5", 1))
            .otherwise(pl.lit("donothing"))
            .alias("treatment_5"),
            pl.when(pl.col("treatment").str.contains("_25"))
            .then(pl.col("treatment").str.extract(r"(\w+)_25", 1))
            .otherwise(pl.lit("donothing"))
            .alias("treatment_25"),
        ]
    )

    df = df_keys.join(df, on=["unit", "schedule"], how="inner")

    return df


def adjust_data(base_df: pl.DataFrame, df: pl.DataFrame, discount_factor: int = 5) -> pl.DataFrame:
    df = df.rename({"harvest_value_5": "harvest_value_old"})
    df = df.with_columns(
        [
            base_df["harvest_value_5"].alias("harvest_value_5"),
            (
                df["npv_5_percent"]
                - (df["harvest_value_old"] + base_df["harvest_value_5"]) * (1 - discount_factor / 100) ** 5
            ).alias("npv_5_percent"),
        ]
    )
    df=df.drop("harvest_value_old")
    return df

In [ ]:
discount_factor = 5
data_dir = "C:/MyTemp/data/two_stage_data/l1l23"
base_df = format_data(data_dir=data_dir,discount_factor=discount_factor)
base_df.write_csv(Path(f"{data_dir}/scenario.csv"))

In [ ]:
discount_factor = 5
data_dir = "C:/MyTemp/data/two_stage_data/h1h23"
df = format_data(data_dir=data_dir,discount_factor=discount_factor)
df = adjust_data(base_df=base_df, df=df,discount_factor=discount_factor)

groups = df.group_by(["unit", "treatment_5"])

adjusted_groups = []
# Now loop through each group
for unit, group_df in groups:
    donothing = group_df.filter(pl.col("treatment_25") == "donothing")

    adjusted_df = group_df

    if donothing.height > 0:
        # Grab the first (and probably only) row
        stock_1_5 = donothing["stock_1_5"][0]
        stock_1_25 = donothing["stock_1_25"][0]
        stock_25 = donothing["stock_25"][0]

        # Calculate the difference
        difference = stock_1_25 - stock_1_5
        multiplier = (difference / stock_25) / 2
        vmultiplier = (stock_1_25 / stock_25) / 2

        # Store original harvest_value_25 as a temp column
        adjusted_df = adjusted_df.with_columns(pl.col("harvest_value_25").alias("original_harvest_value_25"))

        # Apply multiplier to stock_25 and harvest_value_25
        adjusted_df = adjusted_df.with_columns(
            [
                (pl.col("stock_25") * (1 - multiplier)).alias("stock_25"),
                (pl.col("harvest_value_25") * (1 - multiplier)).alias("harvest_value_25"),
            ]
        )

        # Subtract the per-row change
        adjusted_df = adjusted_df.with_columns(
            (
                pl.col(f"npv_{discount_factor}_percent")
                - (pl.col("original_harvest_value_25") - pl.col("harvest_value_25")) * (1 - discount_factor / 100) ** 25
            ).alias(f"npv_{discount_factor}_percent")
        )

        # drop the temp column
        adjusted_df = adjusted_df.drop("original_harvest_value_25")
    else:
        print("No 'donothing' row found in this group")
    adjusted_groups.append(adjusted_df)

new_df = pl.concat(adjusted_groups)
new_df = new_df.sort(["unit", "schedule"])

new_df.write_csv(Path(f"{data_dir}/scenario.csv"))

In [ ]:
discount_factor = 5
data_dir = "C:/MyTemp/data/two_stage_data/l1h23"

df = format_data(data_dir=data_dir,discount_factor=discount_factor)
df = adjust_data(base_df=base_df, df=df,discount_factor=discount_factor)

# Group by "unit"
groups = df.group_by(["unit", "treatment_5"])

adjusted_groups = []
# Now loop through each group
for unit, group_df in groups:
    donothing = group_df.filter(pl.col("treatment_25") == "donothing")

    adjusted_df = group_df

    if donothing.height > 0:
        # Grab the first (and probably only) row
        stock_1_5 = donothing["stock_1_5"][0]
        stock_1_25 = donothing["stock_1_25"][0]
        stock_30_5 = donothing["stock_30_5"][0]
        stock_30_25 = donothing["stock_30_25"][0]
        stock_25 = donothing["stock_25"][0]

        # Calculate the difference
        difference = stock_1_25 + stock_30_25 - stock_1_5 - stock_30_5
        multiplier = (difference / stock_25) / 5

        # Store original harvest_value_25 as a temp column
        adjusted_df = adjusted_df.with_columns(pl.col("harvest_value_25").alias("original_harvest_value_25"))

        # Apply multiplier to stock_25 and harvest_value_25
        adjusted_df = adjusted_df.with_columns(
            [
                (pl.col("stock_25") * (1 - multiplier)).alias("stock_25"),
                (pl.col("harvest_value_25") * (1 - multiplier)).alias("harvest_value_25"),
            ]
        )

        # Subtract the per-row change
        adjusted_df = adjusted_df.with_columns(
            (
                pl.col(f"npv_{discount_factor}_percent")
                - (pl.col("original_harvest_value_25") - pl.col("harvest_value_25")) * (1 - discount_factor / 100) ** 25
            ).alias(f"npv_{discount_factor}_percent")
        )

        # drop the temp column
        adjusted_df = adjusted_df.drop("original_harvest_value_25")
        # print(f"Difference for donothing row: {difference}")
        # print(f"multiplier: {multiplier}")
    else:
        print("No 'donothing' row found in this group")
    adjusted_groups.append(adjusted_df)

new_df = pl.concat(adjusted_groups)
new_df = new_df.sort(["unit", "schedule"])
new_df.write_csv(Path(f"{data_dir}/scenario.csv"))

In [ ]:
discount_factor = 5
data_dir = "C:/MyTemp/data/two_stage_data/h1l23"

df = format_data(data_dir=data_dir,discount_factor=discount_factor)
df = adjust_data(base_df=base_df, df=df,discount_factor=discount_factor)

# Group by "unit"
groups = df.group_by(["unit", "treatment_5"])

adjusted_groups = []
# Now loop through each group
for unit, group_df in groups:
    donothing = group_df.filter(pl.col("treatment_25") == "donothing")

    adjusted_df = group_df

    if donothing.height > 0:
        # Grab the first (and probably only) row
        stock_2_5 = donothing["stock_2_5"][0]
        stock_2_25 = donothing["stock_2_25"][0]
        stock_25 = donothing["stock_25"][0]

        # Calculate the difference
        difference = stock_2_25 - stock_2_5
        multiplier = (difference / stock_25) / 10

        # Store original harvest_value_25 as a temp column
        adjusted_df = adjusted_df.with_columns(pl.col("harvest_value_25").alias("original_harvest_value_25"))

        # Apply multiplier to stock_25 and harvest_value_25
        adjusted_df = adjusted_df.with_columns(
            [
                (pl.col("stock_25") * (1 - multiplier)).alias("stock_25"),
                (pl.col("harvest_value_25") * (1 - multiplier)).alias("harvest_value_25"),
            ]
        )

        # Subtract the per-row change
        adjusted_df = adjusted_df.with_columns(
            (
                pl.col(f"npv_{discount_factor}_percent")
                - (pl.col("original_harvest_value_25") - pl.col("harvest_value_25")) * (1 - discount_factor / 100) ** 25
            ).alias(f"npv_{discount_factor}_percent")
        )

        # drop the temp column
        adjusted_df = adjusted_df.drop("original_harvest_value_25")
        # print(f"Difference for donothing row: {difference}")
        # print(f"multiplier: {multiplier}")
    else:
        print("No 'donothing' row found in this group")
    adjusted_groups.append(adjusted_df)

new_df = pl.concat(adjusted_groups)
new_df = new_df.sort(["unit", "schedule"])
new_df.write_csv(Path(f"{data_dir}/scenario.csv"))

In [ ]:
discount_factor = 5
data_dir = "C:/MyTemp/data/two_stage_data/h1h23"
df = pl.read_csv(Path(f"{data_dir}/alternatives.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000)
df = df.select(
    [
        "unit",
        "schedule",
        f"npv_{discount_factor}_percent",
        "stock_0",
        "stock_5",
        "stock_25",
        "harvest_value_5",
        "harvest_value_25",
        "stock_1_0",
        "stock_1_5",
        "stock_1_25",
        "stock_2_0",
        "stock_2_5",
        "stock_2_25",
        "stock_30_0",
        "stock_30_5",
        "stock_30_25",
    ]
)
df_keys = pl.read_csv(
    Path(f"{data_dir}/alternatives_key.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000
)
df_keys = df_keys.drop("holding")
# Split the treatments into multiple columns, to make them easier to read or whatever
df_keys = df_keys.with_columns(
    [
        pl.when(pl.col("treatment").str.contains("_5"))
        .then(pl.col("treatment").str.extract(r"(\w+)_5", 1))
        .otherwise(pl.lit("donothing"))
        .alias("treatment_5"),
        pl.when(pl.col("treatment").str.contains("_25"))
        .then(pl.col("treatment").str.extract(r"(\w+)_25", 1))
        .otherwise(pl.lit("donothing"))
        .alias("treatment_25"),
    ]
)

df = df_keys.join(df, on=["unit", "schedule"], how="inner")


In [ ]:
# pl.Config.set_tbl_rows(100)
# df

In [23]:
# Group by "unit"
groups = df.group_by(["unit", "treatment_5"])

adjusted_groups = []
# Now loop through each group
for unit, group_df in groups:
    donothing = group_df.filter(pl.col("treatment_25") == "donothing")

    adjusted_df = group_df

    if donothing.height > 0:
        # Grab the first (and probably only) row
        stock_1_5 = donothing["stock_1_5"][0]
        stock_1_25 = donothing["stock_1_25"][0]
        stock_25 = donothing["stock_25"][0]

        # Calculate the difference
        difference = stock_1_25 - stock_1_5
        multiplier = (difference / stock_25) / 2
        vmultiplier = (stock_1_25 / stock_25) / 2

        # Store original harvest_value_25 as a temp column
        adjusted_df = adjusted_df.with_columns(pl.col("harvest_value_25").alias("original_harvest_value_25"))

        # Apply multiplier to stock_25 and harvest_value_25
        adjusted_df = adjusted_df.with_columns(
            [
                (pl.col("stock_25") * (1 - multiplier)).alias("stock_25"),
                (pl.col("harvest_value_25") * (1 - multiplier)).alias("harvest_value_25"),
            ]
        )

        # Subtract the per-row change
        adjusted_df = adjusted_df.with_columns(
            (
                pl.col(f"npv_{discount_factor}_percent")
                - (pl.col("original_harvest_value_25") - pl.col("harvest_value_25")) * (1 - discount_factor / 100) ** 25
            ).alias(f"npv_{discount_factor}_percent")
        )

        # drop the temp column
        adjusted_df = adjusted_df.drop("original_harvest_value_25")
    else:
        print("No 'donothing' row found in this group")
    adjusted_groups.append(adjusted_df)

new_df = pl.concat(adjusted_groups)
new_df = new_df.sort(["unit", "schedule"])

In [ ]:
#new_df.write_csv(Path(f"{data_dir}/scenario.csv"))

In [ ]:
discount_factor = 5
data_dir = "C:/MyTemp/data/two_stage_data/l1l23"
df = pl.read_csv(Path(f"{data_dir}/alternatives.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000)
df = df.select(
    [
        "unit",
        "schedule",
        f"npv_{discount_factor}_percent",
        "stock_0",
        "stock_5",
        "stock_25",
        "harvest_value_5",
        "harvest_value_25",
        "stock_1_0",
        "stock_1_5",
        "stock_1_25",
        "stock_2_0",
        "stock_2_5",
        "stock_2_25",
        "stock_30_0",
        "stock_30_5",
        "stock_30_25",
    ]
)
df_keys = pl.read_csv(
    Path(f"{data_dir}/alternatives_key.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000
)
df_keys = df_keys.drop("holding")
# Split the treatments into multiple columns, to make them easier to read or whatever
df_keys = df_keys.with_columns(
    [
        pl.when(pl.col("treatment").str.contains("_5"))
        .then(pl.col("treatment").str.extract(r"(\w+)_5", 1))
        .otherwise(pl.lit("donothing"))
        .alias("treatment_5"),
        pl.when(pl.col("treatment").str.contains("_25"))
        .then(pl.col("treatment").str.extract(r"(\w+)_25", 1))
        .otherwise(pl.lit("donothing"))
        .alias("treatment_25"),
    ]
)

df = df_keys.join(df, on=["unit", "schedule"], how="inner")
#df.write_csv(Path(f"{data_dir}/scenario.csv"))

In [26]:
discount_factor = 5
data_dir = "C:/MyTemp/data/two_stage_data/l1h23"
df = pl.read_csv(Path(f"{data_dir}/alternatives.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000)
df = df.select(
    [
        "unit",
        "schedule",
        f"npv_{discount_factor}_percent",
        "stock_0",
        "stock_5",
        "stock_25",
        "harvest_value_5",
        "harvest_value_25",
        "stock_1_0",
        "stock_1_5",
        "stock_1_25",
        "stock_2_0",
        "stock_2_5",
        "stock_2_25",
        "stock_30_0",
        "stock_30_5",
        "stock_30_25",
    ]
)
df_keys = pl.read_csv(
    Path(f"{data_dir}/alternatives_key.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000
)
df_keys = df_keys.drop("holding")
# Split the treatments into multiple columns, to make them easier to read or whatever
df_keys = df_keys.with_columns(
    [
        pl.when(pl.col("treatment").str.contains("_5"))
        .then(pl.col("treatment").str.extract(r"(\w+)_5", 1))
        .otherwise(pl.lit("donothing"))
        .alias("treatment_5"),
        pl.when(pl.col("treatment").str.contains("_25"))
        .then(pl.col("treatment").str.extract(r"(\w+)_25", 1))
        .otherwise(pl.lit("donothing"))
        .alias("treatment_25"),
    ]
)

df = df_keys.join(df, on=["unit", "schedule"], how="inner")


In [ ]:
# Group by "unit"
groups = df.group_by(["unit", "treatment_5"])

adjusted_groups = []
# Now loop through each group
for unit, group_df in groups:
    donothing = group_df.filter(pl.col("treatment_25") == "donothing")

    adjusted_df = group_df

    if donothing.height > 0:
        # Grab the first (and probably only) row
        stock_1_5 = donothing["stock_1_5"][0]
        stock_1_25 = donothing["stock_1_25"][0]
        stock_30_5 = donothing["stock_30_5"][0]
        stock_30_25 = donothing["stock_30_25"][0]
        stock_25 = donothing["stock_25"][0]

        # Calculate the difference
        difference = stock_1_25 + stock_30_25 - stock_1_5 - stock_30_5
        multiplier = (difference / stock_25) / 5

        # Store original harvest_value_25 as a temp column
        adjusted_df = adjusted_df.with_columns(pl.col("harvest_value_25").alias("original_harvest_value_25"))

        # Apply multiplier to stock_25 and harvest_value_25
        adjusted_df = adjusted_df.with_columns(
            [
                (pl.col("stock_25") * (1 - multiplier)).alias("stock_25"),
                (pl.col("harvest_value_25") * (1 - multiplier)).alias("harvest_value_25"),
            ]
        )

        # Subtract the per-row change
        adjusted_df = adjusted_df.with_columns(
            (
                pl.col(f"npv_{discount_factor}_percent")
                - (pl.col("original_harvest_value_25") - pl.col("harvest_value_25")) * (1 - discount_factor / 100) ** 25
            ).alias(f"npv_{discount_factor}_percent")
        )

        # drop the temp column
        adjusted_df = adjusted_df.drop("original_harvest_value_25")
        # print(f"Difference for donothing row: {difference}")
        # print(f"multiplier: {multiplier}")
    else:
        print("No 'donothing' row found in this group")
    adjusted_groups.append(adjusted_df)

new_df = pl.concat(adjusted_groups)
new_df = new_df.sort(["unit", "schedule"])
#new_df.write_csv(Path(f"{data_dir}/scenario.csv"))

In [28]:
discount_factor = 5
data_dir = "C:/MyTemp/data/two_stage_data/h1l23"
df = pl.read_csv(Path(f"{data_dir}/alternatives.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000)
df = df.select(
    [
        "unit",
        "schedule",
        f"npv_{discount_factor}_percent",
        "stock_0",
        "stock_5",
        "stock_25",
        "harvest_value_5",
        "harvest_value_25",
        "stock_1_0",
        "stock_1_5",
        "stock_1_25",
        "stock_2_0",
        "stock_2_5",
        "stock_2_25",
        "stock_30_0",
        "stock_30_5",
        "stock_30_25",
    ]
)
df_keys = pl.read_csv(
    Path(f"{data_dir}/alternatives_key.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000
)
df_keys = df_keys.drop("holding")
# Split the treatments into multiple columns, to make them easier to read or whatever
df_keys = df_keys.with_columns(
    [
        pl.when(pl.col("treatment").str.contains("_5"))
        .then(pl.col("treatment").str.extract(r"(\w+)_5", 1))
        .otherwise(pl.lit("donothing"))
        .alias("treatment_5"),
        pl.when(pl.col("treatment").str.contains("_25"))
        .then(pl.col("treatment").str.extract(r"(\w+)_25", 1))
        .otherwise(pl.lit("donothing"))
        .alias("treatment_25"),
    ]
)

df = df_keys.join(df, on=["unit", "schedule"], how="inner")

In [ ]:
# Group by "unit"
groups = df.group_by(["unit", "treatment_5"])

adjusted_groups = []
# Now loop through each group
for unit, group_df in groups:
    donothing = group_df.filter(pl.col("treatment_25") == "donothing")

    adjusted_df = group_df

    if donothing.height > 0:
        # Grab the first (and probably only) row
        stock_2_5 = donothing["stock_2_5"][0]
        stock_2_25 = donothing["stock_2_25"][0]
        stock_25 = donothing["stock_25"][0]

        # Calculate the difference
        difference = stock_2_25 - stock_2_5
        multiplier = (difference / stock_25) / 10

        # Store original harvest_value_25 as a temp column
        adjusted_df = adjusted_df.with_columns(pl.col("harvest_value_25").alias("original_harvest_value_25"))

        # Apply multiplier to stock_25 and harvest_value_25
        adjusted_df = adjusted_df.with_columns(
            [
                (pl.col("stock_25") * (1 - multiplier)).alias("stock_25"),
                (pl.col("harvest_value_25") * (1 - multiplier)).alias("harvest_value_25"),
            ]
        )

        # Subtract the per-row change
        adjusted_df = adjusted_df.with_columns(
            (
                pl.col(f"npv_{discount_factor}_percent")
                - (pl.col("original_harvest_value_25") - pl.col("harvest_value_25")) * (1 - discount_factor / 100) ** 25
            ).alias(f"npv_{discount_factor}_percent")
        )

        # drop the temp column
        adjusted_df = adjusted_df.drop("original_harvest_value_25")
        # print(f"Difference for donothing row: {difference}")
        # print(f"multiplier: {multiplier}")
    else:
        print("No 'donothing' row found in this group")
    adjusted_groups.append(adjusted_df)

new_df = pl.concat(adjusted_groups)
new_df = new_df.sort(["unit", "schedule"])
#new_df.write_csv(Path(f"{data_dir}/scenario.csv"))